In [41]:
import pandas as pd

In [42]:
!pip install openpyxl
!pip install transformers sentencepiece sacremoses rouge-score

In [43]:
df = pd.read_excel('NewsSumm Dataset.xlsx')

In [44]:
df.head()

,newspaper_name,published_date\n,headline,article_text,human_summary,news_category
0,Indian Express,2020-06-01 00:00:00,Virus may be invisible enemy but COVID warrior...,Prime Minister Narendra Modi Monday hailed the...,Prime Minister of India said that the Virus ma...,National News
1,Economic Times,2013-02-11 00:00:00,"Economy can bounce back, says PM Modi","ALLAHABAD: At least 20 persons were killed, an...","In Maha Kumbh, nearly 20 persons were killed. ...",National News
2,Business Standard,2013-02-11 00:00:00,At least 20 killed in stampede in Allahabad,"At least 20 people were killed, and scores of ...",As per the sources 20 people died and scores w...,National News
3,Money Control,2013-02-11 00:00:00,Maha Kumbh: Over 20 dead in Allahabad station ...,More than 20 people were feared dead and 30 ot...,At least 20 people killed and 20 people are in...,National News
4,The Mint,2023-10-02 00:00:00,Gandhian wisdom,"This Gandhi Jayanti, we should reflect upon an...","In this article, the author reflects on Mahatm...",National News


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348766 entries, 0 to 348765
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   newspaper_name   348762 non-null  object
 1   published_date
  348762 non-null  object
 2   headline         348761 non-null  object
 3   article_text     348739 non-null  object
 4   human_summary    348526 non-null  object
 5   news_category    348760 non-null  object
dtypes: object(6)
memory usage: 16.0+ MB


In [46]:
df_small = df.sample(100, random_state=42).reset_index(drop=True)


In [47]:
df_small

,newspaper_name,published_date\n,headline,article_text,human_summary,news_category
0,THE TIMES OF INDIA,2009-12-07 00:00:00,Will Ganga be clean in next 11 yrs?,Attempting to silence traditional scepticism o...,Union Minister Jairam Ramesh pledged a 20-25% ...,Politics
1,Hindustan Times,2020-03-12 00:00:00,ED on Rana Kapoor,"Mumbai : YES Bank, under co-founder Rana Kapoo...",\nThe Enforcement Directorate (ED) has reveale...,Entertainment
2,Hindustan Times,2025-07-08 00:00:00,"Indian nurse Nimisha Priya, sentenced to death...","Nimisha Priya, 37, who originally belongs to K...","Nimisha Priya, a 37-year-old Indian nurse from...",Crime and Justice
3,The Times of India,2008-02-20 00:00:00,Taare Zameen tax-free,CHANDIGARH: Haryana government on Wednesday ex...,The Haryana government exempts 'Taare Zameen P...,Entertainment
4,TIMES OF INDIA,2023-12-18 00:00:00,Chhattisgarh Congress faces wave of resignatio...,RAIPUR: The Chhattisgarh Congress continues to...,Chhattisgarh Congress is grappling with a wave...,Politics
...,...,...,...,...,...,...
95,Financial Times,2025-07-02 00:00:00,Infosys employee arrested for making over 30 w...,"On June 30, a 28‑year‑old Infosys Senior Assoc...","At Infosys’s Bengaluru campus, a 28-year-old c...",National News
96,The pioneer,2018-01-31 00:00:00,HP takes steps to benefit people: CM,Himachal Pradesh Chief Minister Jai Ram Thakur...,Himachal Pradesh Chief Minister Jai Ram Thakur...,Local News
97,The Asian Age,2024-02-27 00:00:00,"Ghazal legend Pankaj Udhas, 72, no more",Udhas was known for his velvety renditions of ...,"Pankaj Udhas, the legendary ghazal singer, pas...",Obituaries
98,Hindustan Times,2018-07-22 00:00:00,Pakistan election: Bilawal Bhutto wins hearts ...,"The chances of Bilawal Bhutto, chairman of the...","\nBilawal Bhutto, chairman of the Pakistan Peo...",National News


In [48]:
from transformers import pipeline

hi_en = pipeline(
    "translation",
    model="Helsinki-NLP/opus-mt-hi-en"
)

Device set to use cpu


In [49]:
en_hi = pipeline(
    "translation",
    model="Helsinki-NLP/opus-mt-en-hi"
)

Device set to use cpu


In [50]:
def truncate_text(text, max_chars=2000):
    if not isinstance(text, str):
        return ""
    return text[:max_chars]


In [51]:
def translate_en_to_hi(text):
    text = truncate_text(text, max_chars=2000)
    output = en_hi(
        text,
        max_length=512,
        truncation=True
    )
    return output[0]["translation_text"]


In [61]:
def chunk_text(text, max_chars=800):
    return [text[i:i+max_chars] for i in range(0, len(text), max_chars)]


In [62]:
def translate_en_to_hi_full(text):
    chunks = chunk_text(text)
    translated_chunks = []

    for chunk in chunks:
        out = en_hi(chunk, max_length=512, truncation=True)
        translated_chunks.append(out[0]["translation_text"])

    return " ".join(translated_chunks)


In [63]:
df_small["article_hi"] = df_small["article_text"].apply(translate_en_to_hi_full)
 

In [65]:
df_small

,newspaper_name,published_date\n,headline,article_text,human_summary,news_category,article_text_hindi,article_hi
0,THE TIMES OF INDIA,2009-12-07 00:00:00,Will Ganga be clean in next 11 yrs?,Attempting to silence traditional scepticism o...,Union Minister Jairam Ramesh pledged a 20-25% ...,Politics,"""विद्योगिकियों के लिए चिकित्सा की गंभीरता"" की ...",भारत के पर्यावरण मुद्दों की गंभीरता को शांत कर...
1,Hindustan Times,2020-03-12 00:00:00,ED on Rana Kapoor,"Mumbai : YES Bank, under co-founder Rana Kapoo...",\nThe Enforcement Directorate (ED) has reveale...,Entertainment,"एम. डी.) के कानून के तहत, मैड्‌स के कानून के ब...","मुंबई बैंक के तहत : . Wacaka बैंक के तहत, ऋण क..."
2,Hindustan Times,2025-07-08 00:00:00,"Indian nurse Nimisha Priya, sentenced to death...","Nimisha Priya, 37, who originally belongs to K...","Nimisha Priya, a 37-year-old Indian nurse from...",Crime and Justice,उसने कहा कि इस कानून के बारे में मेड्रिड मे मे...,"नीमीना प्रिया, जो वास्तव में केरल का है, दीओला..."
3,The Times of India,2008-02-20 00:00:00,Taare Zameen tax-free,CHANDIGARH: Haryana government on Wednesday ex...,The Haryana government exempts 'Taare Zameen P...,Entertainment,SERIAGH: बुधवार पर हराना सरकार ने Amamamamamor...,SERIAGH: बुधवार पर हराना सरकार ने Amamamamamor...
4,TIMES OF INDIA,2023-12-18 00:00:00,Chhattisgarh Congress faces wave of resignatio...,RAIPUR: The Chhattisgarh Congress continues to...,Chhattisgarh Congress is grappling with a wave...,Politics,कानून कानून कानून के तहत कानून के तहत कानून के...,RARANR: Thrgagsagacacaps की एक हत्या का सामना ...
...,...,...,...,...,...,...,...,...
95,Financial Times,2025-07-02 00:00:00,Infosys employee arrested for making over 30 w...,"On June 30, a 28‑year‑old Infosys Senior Assoc...","At Infosys’s Bengaluru campus, a 28-year-old c...",National News,"जून 30, 30 को 28 साल की एक लड़की से मुलाकात की...","जून 30, 30 को 28 साल की एक लड़की से मुलाकात की..."
96,The pioneer,2018-01-31 00:00:00,HP takes steps to benefit people: CM,Himachal Pradesh Chief Minister Jai Ram Thakur...,Himachal Pradesh Chief Minister Jai Ram Thakur...,Local News,उसने कहा कि उस कानून के तहत जो कानून तैयार किय...,इस बारे में पिछले एक महीने में सरकार ने लोगों ...
97,The Asian Age,2024-02-27 00:00:00,"Ghazal legend Pankaj Udhas, 72, no more",Udhas was known for his velvety renditions of ...,"Pankaj Udhas, the legendary ghazal singer, pas...",Obituaries,उसने कहा: “मैं अपनी बीमारी के बारे में बहुत ही...,"Uddwids अपने परिवार के लिए जाना जाता था, खास त..."
98,Hindustan Times,2018-07-22 00:00:00,Pakistan election: Bilawal Bhutto wins hearts ...,"The chances of Bilawal Bhutto, chairman of the...","\nBilawal Bhutto, chairman of the Pakistan Peo...",National News,इस वज़ह से वह अपने वकील के पास गया है कि उसके ...,"पाकिस्तान के लोगों की पार्टी के अध्यक्ष, बिस्च..."


In [68]:
df_small = df_small.drop(columns=["article_text_hindi"])


In [69]:
df_small

,newspaper_name,published_date\n,headline,article_text,human_summary,news_category,article_hi
0,THE TIMES OF INDIA,2009-12-07 00:00:00,Will Ganga be clean in next 11 yrs?,Attempting to silence traditional scepticism o...,Union Minister Jairam Ramesh pledged a 20-25% ...,Politics,भारत के पर्यावरण मुद्दों की गंभीरता को शांत कर...
1,Hindustan Times,2020-03-12 00:00:00,ED on Rana Kapoor,"Mumbai : YES Bank, under co-founder Rana Kapoo...",\nThe Enforcement Directorate (ED) has reveale...,Entertainment,"मुंबई बैंक के तहत : . Wacaka बैंक के तहत, ऋण क..."
2,Hindustan Times,2025-07-08 00:00:00,"Indian nurse Nimisha Priya, sentenced to death...","Nimisha Priya, 37, who originally belongs to K...","Nimisha Priya, a 37-year-old Indian nurse from...",Crime and Justice,"नीमीना प्रिया, जो वास्तव में केरल का है, दीओला..."
3,The Times of India,2008-02-20 00:00:00,Taare Zameen tax-free,CHANDIGARH: Haryana government on Wednesday ex...,The Haryana government exempts 'Taare Zameen P...,Entertainment,SERIAGH: बुधवार पर हराना सरकार ने Amamamamamor...
4,TIMES OF INDIA,2023-12-18 00:00:00,Chhattisgarh Congress faces wave of resignatio...,RAIPUR: The Chhattisgarh Congress continues to...,Chhattisgarh Congress is grappling with a wave...,Politics,RARANR: Thrgagsagacacaps की एक हत्या का सामना ...
...,...,...,...,...,...,...,...
95,Financial Times,2025-07-02 00:00:00,Infosys employee arrested for making over 30 w...,"On June 30, a 28‑year‑old Infosys Senior Assoc...","At Infosys’s Bengaluru campus, a 28-year-old c...",National News,"जून 30, 30 को 28 साल की एक लड़की से मुलाकात की..."
96,The pioneer,2018-01-31 00:00:00,HP takes steps to benefit people: CM,Himachal Pradesh Chief Minister Jai Ram Thakur...,Himachal Pradesh Chief Minister Jai Ram Thakur...,Local News,इस बारे में पिछले एक महीने में सरकार ने लोगों ...
97,The Asian Age,2024-02-27 00:00:00,"Ghazal legend Pankaj Udhas, 72, no more",Udhas was known for his velvety renditions of ...,"Pankaj Udhas, the legendary ghazal singer, pas...",Obituaries,"Uddwids अपने परिवार के लिए जाना जाता था, खास त..."
98,Hindustan Times,2018-07-22 00:00:00,Pakistan election: Bilawal Bhutto wins hearts ...,"The chances of Bilawal Bhutto, chairman of the...","\nBilawal Bhutto, chairman of the Pakistan Peo...",National News,"पाकिस्तान के लोगों की पार्टी के अध्यक्ष, बिस्च..."


In [73]:
df_small.to_csv("newsumm_crosslingual_100.csv", index=False)


In [95]:
df_small = df_small.drop(columns=["pred_summary_hi_en"])

In [96]:
df_small.head(2)

,newspaper_name,published_date\n,headline,article_text,human_summary,news_category,article_hi
0,THE TIMES OF INDIA,2009-12-07 00:00:00,Will Ganga be clean in next 11 yrs?,Attempting to silence traditional scepticism o...,Union Minister Jairam Ramesh pledged a 20-25% ...,Politics,भारत के पर्यावरण मुद्दों की गंभीरता को शांत कर...
1,Hindustan Times,2020-03-12 00:00:00,ED on Rana Kapoor,"Mumbai : YES Bank, under co-founder Rana Kapoo...",\nThe Enforcement Directorate (ED) has reveale...,Entertainment,"मुंबई बैंक के तहत : . Wacaka बैंक के तहत, ऋण क..."


In [97]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

tokenizer = MBart50TokenizerFast.from_pretrained(
    "facebook/mbart-large-50-many-to-many-mmt"
)

model = MBartForConditionalGeneration.from_pretrained(
    "facebook/mbart-large-50-many-to-many-mmt"
)

# Source language = Hindi
tokenizer.src_lang = "hi_IN"


c:\Users\shash\.conda\envs\flaskenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shash\.cache\huggingface\hub\models--facebook--mbart-large-50-many-to-many-mmt. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling bac

In [98]:


def hindi_to_english_summary(text):
    text = str(text).strip()

    if len(text) < 50:
        return ""

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    )

    summary_ids = model.generate(
        **inputs,
        max_length=200,
        num_beams=4,
        forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
    )

    return tokenizer.decode(
        summary_ids[0],
        skip_special_tokens=True
    )


In [99]:
df_small["pred_summary_hi_en"] = df_small["article_hi"].apply(
    hindi_to_english_summary
)


In [102]:
df_small['pred_summary_hi_en'].iloc[0]

'Trying to calm down the seriousness of India \'s environmental issues, Federal Minister Sham Ramesh guaranteed its carbon intensity distribution over the next 11 years. It is enthusiastic to see the Minister trying to create an objective for carbon emissions worldwide and larger, and in all countries for India\' s major projections, so much more. "As far as that \'s a little more success," thinks of a Secraphraphating, and a little more than a little. "It\' s a little more than a little more than a little more. Pollution?" If it isn \'t, here are some data that can help him and the government to come up with a strategy and bring back these two rivers, which are probably for health in 11 years. A person realizes that it was actually called for a sacred jail for a Delhi when he was actually in a sacred jail, and more than three years'

In [103]:
df_small.to_csv("Final_newsumm_crosslingual_100.csv", index=False)

In [118]:
i = 56
print("Hindi Article:\n", df_small.loc[i, "article_hi"])
print("\nGenerated English Summary:\n", df_small.loc[i, "pred_summary_hi_en"])
print("\nGold English Summary:\n", df_small.loc[i, "human_summary"])


Hindi Article:
 AAP के पाइप TJB के एक हाथ में, दिल्ली के JalB बोर्ड (JBB) एक रूपरेखा पूरा करने के लिए व्यस्त है AAPCANDC के लक्ष्य पर, यह अपने आप को एक मजबूत स्थिति में डाल दिया है swawawww. हर महीने के लिए एक महत्वपूर्ण मामले के लिए तैयार किया गया था, लेकिन हर महीने के-दूसरे से कम से कम एक मशीन के साथ एक घंटे के लिए तैयार किया गया था।

Generated English Summary:
 AAP Pipe TJB in one hand, Delhi 's JalB Board (JBB) is busy to complete a plan on AAPCANDC' s goal, it has put itself in a strong position swawawww. Every month was prepared for an important matter, but every month - - at least one machine with each other was prepared for an hour.

Gold English Summary:
 The Delhi Jal Board (DJB) is facing a dilemma as it works on fulfilling the AAP Government's goal of providing "free lifeline water" to residents. Chief Minister Arvind Kejriwal is opposed to the mandatory annual tariff hike of 10 percent, creating a contentious issue. A board meeting scheduled to address this matter was defe

In [108]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeL"],
    use_stemmer=True
)

def rouge_l(ref, pred):
    return scorer.score(ref, pred)["rougeL"].fmeasure

df_small["rougeL_hi_en"] = df_small.apply(
    lambda x: rouge_l(x["human_summary"], x["pred_summary_hi_en"]),
    axis=1
)

print("Average ROUGE-L (Hindi → English):",
      df_small["rougeL_hi_en"].mean())


Average ROUGE-L (Hindi → English): 0.1763112475324963
